In [1]:
# Running %env without any arguments
# lists all environment variables

# The line below sets the environment
# variable CUDA_VISIBLE_DEVICES
%env CUDA_VISIBLE_DEVICES = 0

import io
import time
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from scipy.misc import imread, imsave
import tensorflow as tf
from tensorflow.python.platform import tf_logging
import os.path
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.python.slim.nets import inception
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
import inception_preprocessing
import logging

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

env: CUDA_VISIBLE_DEVICES=0


In [2]:
DATASET_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/'
LOG_PATH = DATASET_PATH + 'test_log/'
TEST_PATH = DATASET_PATH + 'Test/'
OUTPUT_PATH = DATASET_PATH + 'output_{}.csv'
CATEGORY_NAME_PATH = DATASET_PATH + 'category_names.csv'
INCEPTION_MODEL_PATH = DATASET_PATH + 'test_log/inception_v3_model.ckpt-463613.data-00000-of-00001'
BATCH_SIZE = 256
IMAGE_WIDTH = 180
IMAGE_HEIGHT = 180
NUM_CLASS = 5270
TOTAL_EXAMPLES = 3095080
INPUT_THREADS = 1
NUM_STEPS = int(TOTAL_EXAMPLES / BATCH_SIZE) + 1

In [ ]:
# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# create file handler which logs even debug messages
fh = logging.FileHandler(DATASET_PATH + 'tensorflow_inception_test.log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
log.addHandler(fh)

In [3]:
class MiniDataSet(object):
    def __init__(self, file_path_pattern, num_examples, num_classes, is_training = True, min_after_dequeue=1000, batch_size = BATCH_SIZE, num_reader = INPUT_THREADS):
        super(MiniDataSet, self).__init__()
        self._num_examples = num_examples
        self._num_classes = num_classes
        self._file_path_pattern = file_path_pattern
        self._num_reader = num_reader
        self._batch_size = batch_size
        self._min_after_dequeue = min_after_dequeue
        self._is_training = is_training
        
    def create_dataset(self):
        opts = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.ZLIB)
        reader = lambda : tf.TFRecordReader(options=opts)
        keys_to_features = {
            'product_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
            'img_raw': tf.FixedLenFeature([], tf.string, default_value=''),
            # notice that we don't have this feature in our TFRecord, so always default provided
            'format': tf.FixedLenFeature([], tf.string, default_value='jpg')
        }

        items_to_handlers = {
            # automated decode image from features in FixedLenFeature
            'image': slim.tfexample_decoder.Image(image_key='img_raw', format_key='format'),
            'product': slim.tfexample_decoder.Tensor('product_id'),
        }

        decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

        self._dataset = slim.dataset.Dataset(
            data_sources = self._file_path_pattern,
            decoder = decoder,
            reader = reader,
            # num_readers = 8,
            num_samples = self._num_examples,
            #num_classes = self._num_classes,
            items_to_descriptions = None)
        
        # notice that DatasetDataProvider can automate shuffle the examples by ParallelReader using its RandomShuffleQueue
        self._data_provider = slim.dataset_data_provider.DatasetDataProvider(
            self._dataset,
            num_readers = self._num_reader,
            shuffle = False, # default is True
            num_epochs = 1,
            common_queue_capacity = self._min_after_dequeue + 3 * self._batch_size,
            common_queue_min = self._min_after_dequeue,
            scope = 'test_files')
        
        return self._data_provider.get(['image', 'product'])

In [4]:
def_graph = tf.Graph()
with def_graph.as_default() as graph:
    def test_step(input_examples):   
        with slim.arg_scope(inception.inception_v3_arg_scope()):
            # here logits is the pre-softmax activations
            logits, end_points = inception.inception_v3(
                input_examples,
                num_classes = NUM_CLASS,
                is_training=False)
            variables_to_restore = slim.get_variables_to_restore()
            #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
            predictions = tf.argmax(end_points['Predictions'], 1)
            probabilities = end_points['Predictions']

            return predictions, probabilities, variables_to_restore
    def preprocess_for_inception(input_image):
        # inception_v3.default_image_size = 299
        return inception_preprocessing.preprocess_image(input_image, 299, 299, False)
    def init_dataset(file_path_pattern):
        dataset = MiniDataSet(file_path_pattern, TOTAL_EXAMPLES, NUM_CLASS)
        org_image, product_id = dataset.create_dataset()
        image = preprocess_for_inception(org_image)
        batch_images, batch_id = tf.train.batch([image, product_id], BATCH_SIZE,\
                                            num_threads = INPUT_THREADS,\
                                            capacity = 1000 + 3 * BATCH_SIZE,\
                                            allow_smaller_final_batch = True, name = 'test_batch')
        
        return batch_images, batch_id
    def cvt_csv2tfrecord():
        count = 0
        category_map = dict()
        csv = pd.read_csv(CATEGORY_NAME_PATH).values
        for row in csv:  
            category_id, _ = row[0], row[1:]
            category_map[category_id] = count
            count += 1
        return category_map

In [5]:
with def_graph.as_default() as graph:
    mapping_strings = tf.constant( [ str(key) for key in cvt_csv2tfrecord().keys() ] )
    mapping_table = tf.contrib.lookup.index_table_from_tensor(mapping=mapping_strings, default_value=0)
    
    inv_table = tf.contrib.lookup.index_to_string_table_from_tensor(mapping_strings, default_value="0000000000")

    batch_images, batch_id = init_dataset(TEST_PATH + "output_file*.tfrecords")#test_output_file6.tfrecords
    #batch_images, batch_id = init_dataset(TEST_PATH+'test_output_file6*')
    
    # use placeholder instead
    #last_prob = tf.constant(0, shape=[0,NUM_CLASS], dtype=tf.float32)
    #last_id = tf.constant(0, shape=[0], dtype=tf.int64)  
    last_prob = tf.placeholder(tf.float32)
    last_id = tf.placeholder(tf.int64)
    with tf.device('/gpu:0'):
        test_predictions, test_probabilities, variables_to_restore = test_step(batch_images)
        test_predictions_values = tf.string_to_number(inv_table.lookup(test_predictions), out_type=tf.int64) 
        # concat betweent batches
        _, idx, count = tf.unique_with_counts(batch_id)
        #print(tf.dynamic_partition(batch_id, tf.not_equal(idx, tf.shape(count)[0] - 1).eval(), 2)[1].eval())  
        cur_id_tail, _cur_id_head = tf.dynamic_partition(batch_id, tf.cast(tf.not_equal(idx, tf.shape(count)[0] - 1), tf.int32), 2)
        with tf.control_dependencies([cur_id_tail, _cur_id_head]):
            cur_id_head = tf.concat([last_id, _cur_id_head], axis = 0)
        #cur_id_head = tf.concat([last_id, tf.concat(tf.split(batch_id, count)[0:-1], axis = 0)], axis = 0)
        #cur_id_tail = tf.split(batch_id, count)[-1]
        
        cur_prob_tail, _cur_prob_head = tf.dynamic_partition(test_probabilities, tf.cast(tf.not_equal(idx, tf.shape(count)[0] - 1), tf.int32), 2)
        with tf.control_dependencies([last_prob, _cur_prob_head]):
            cur_prob_head = tf.concat([last_prob, _cur_prob_head], axis = 0)
        #cur_prob_head = tf.concat([last_prob, tf.concat(tf.split(test_probabilities, count[0:-1]), axis = 0)], axis = 0)
        #cur_prob_tail = tf.split(test_probabilities, count)[-1]
        with tf.control_dependencies([cur_id_head, cur_prob_head]):
            raw_id, idx, _ = tf.unique_with_counts(cur_id_head)
            mean_prob = tf.segment_mean(cur_prob_head, idx)
            mean_label = tf.string_to_number(inv_table.lookup(tf.argmax(mean_prob, 1)), out_type=tf.int64) 
        with tf.control_dependencies([mean_prob, mean_label]):
            #last_id = cur_id_tail
            #last_prob = cur_prob_tail
            # last partition may have nothing to concat
            raw_id_tail, idx_tail, _ = tf.unique_with_counts(cur_id_tail)
            mean_prob_tail = tf.segment_mean(cur_prob_tail, idx_tail)
            tail_label = tf.string_to_number(inv_table.lookup(tf.argmax(mean_prob_tail, 1)), out_type=tf.int64) 
    restore_saver = tf.train.Saver(variables_to_restore)
    def load_pretrain(sess):
        restore_saver.restore(sess, INCEPTION_MODEL_PATH)

    # no need for specify local_variables_initializer and tables_initializer, Supervisor will do this via default local_init_op
    init_op = tf.group(tf.global_variables_initializer())
    # Pass the init function to the supervisor.
    # - The init function is called _after_ the variables have been initialized by running the init_op.
    # - use default tf.Saver() for ordinary save and restore
    # - save checkpoint every 1.3 hours
    # - manage summary in current process by ourselves for memory saving
    # - no need to specify global_step, supervisor will find this automately
    # - initialize order: checkpoint -> local_init_op -> init_op -> init_func
    sv = tf.train.Supervisor(logdir=LOG_PATH, init_fn = load_pretrain, init_op = init_op, summary_op = None, save_model_secs=0)
    
    step = 0
    lats_pred = []
    last_batch_id = []
    last_feed_id = np.empty([0])
    last_feed_prob = np.empty([0, NUM_CLASS])
    save_file_name= OUTPUT_PATH.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    with sv.managed_session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)) as sess:
    #with sv.prepare_or_wait_for_session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)) as sess:

        # Here sess was either initialized from the pre-trained-checkpoint or
        # recovered from a checkpoint saved in a previous run of this code.
        while True:       
            if sv.should_stop():
                tf_logging.info('Supervisor emit finished!')
                break

            start_time = time.time()
            
            with tf.device('/gpu:0'):
                #test_pred, test_prob, test_batch_id = sess.run([test_predictions_values, test_probabilities, batch_id])
                _, last_feed_id, last_feed_prob, test_pred, test_prob, test_batch_id, lats_pred, last_batch_id = sess.run([batch_id, cur_id_tail, cur_prob_tail, mean_label, mean_prob, raw_id, tail_label, raw_id_tail], feed_dict = {last_prob: last_feed_prob, last_id: last_feed_id})
        
            time_elapsed = time.time() - start_time
            
            if step % 200 == 0:
                tf_logging.info('Step: {} of {}.'.format(step, NUM_STEPS))
                tf_logging.info('Validation Speed: {:5.3f}sec/batch'.format(time_elapsed))
                tf_logging.info('Roughly {:6.3f} hours to go.'.format(  time_elapsed*( (NUM_STEPS-step) > 0 and (NUM_STEPS-step)/3600. or 0.001 )  ))
                tf_logging.info('Test Label: {}'.format(test_pred))
                #tf_logging.info('Test Prob: {}'.format(test_prob))
                tf_logging.info('Test Ids: {}'.format(test_batch_id))
            #print(len(test_prob[0]))
            df = pd.DataFrame({'_id' : test_batch_id, 'category_id' : test_pred})
            #df = pd.DataFrame([test_batch_id, test_pred], columns=["_id", 'category_id'])

            if not os.path.isfile(save_file_name):
                df.to_csv(save_file_name, mode='a', index=False, sep=',')
            else:
                df.to_csv(save_file_name, mode='a', index=False, sep=',', header=False)
            step += 1

#             tf_logging.info('BB ID: {}'.format(bb_id))
#             tf_logging.info('Test Label: {}'.format(test_pred))
#             #tf_logging.info('Test Prob: {}'.format(test_prob))
#             tf_logging.info('Test Ids: {}'.format(test_batch_id))
#             tf_logging.info('Last Label: {}'.format(lats_pred))
#             #tf_logging.info('Test Prob: {}'.format(test_prob))
#             tf_logging.info('Last Ids: {}'.format(last_batch_id))
#             if step > 3:
#                 break
            
    df = pd.DataFrame({'_id' : last_batch_id, 'category_id' : lats_pred})
    df.to_csv(save_file_name, mode='a', index=False, sep=',', header=False)

    tf_logging.info('Finished evaluation! ')
    

INFO:tensorflow:Restoring parameters from /media/rs/0E06CD1706CD0127/Kapok/kaggle/test_log/inception_v3_model.ckpt-463613
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Step: 0 of 12091.
INFO:tensorflow:Validation Speed: 3.136sec/batch
INFO:tensorflow:Roughly 10.532 hours to go.
INFO:tensorflow:Test Label: [1000006181 1000022508 1000005986 1000004079 1000002786 1000004079
 1000010653 1000010653 1000014026 1000016662 1000005874 1000015309
 1000012509 1000010110 1000010653 1000010653 1000006016 1000010667
 1000010653 1000010667 1000020717 1000014281 1000010653 1000015309
 1000010667 1000016662 1000004141 1000011046 1000005414 1000010086
 1000001859 1000003813 1000010653 1000008852 1000013159 1000010653
 1000010153 1000007118 1000002208 1000002548 1000008558 1000010653
 1000010653 1000002597 1000011349 1000004181 1000010090 1000014217
 1000002973 1000000364 1000010653 1000005760 1000017771 1000011566
 1000010683 1000005722 1000018386 10

INFO:tensorflow:Step: 60 of 12091.
INFO:tensorflow:Validation Speed: 1.323sec/batch
INFO:tensorflow:Roughly  4.422 hours to go.
INFO:tensorflow:Test Label: [1000018296 1000021535 1000003684 1000014905 1000004085 1000010116
 1000003787 1000004141 1000003796 1000015452 1000002472 1000012989
 1000015309 1000004079 1000010726 1000018296 1000014197 1000010661
 1000010136 1000005960 1000002443 1000018290 1000003593 1000015662
 1000010170 1000010100 1000005326 1000004079 1000010128 1000011388
 1000010722 1000003787 1000005920 1000018296 1000011427 1000015334
 1000012561 1000004141 1000022209 1000004261 1000000451 1000015309
 1000012993 1000004184 1000018294 1000018294 1000015662 1000010726
 1000001921 1000005874 1000003246 1000010538 1000004141 1000018246
 1000006673 1000008094 1000015763 1000012550 1000004016 1000004141
 1000018296 1000012558 1000016342 1000004079 1000008925 1000012991
 1000006002 1000004079 1000004141 1000018246 1000004079 1000001784
 1000021535 1000006380 1000015912 100001

INFO:tensorflow:Step: 120 of 12091.
INFO:tensorflow:Validation Speed: 1.297sec/batch
INFO:tensorflow:Roughly  4.313 hours to go.
INFO:tensorflow:Test Label: [1000012540 1000016272 1000010663 1000015763 1000008856 1000004085
 1000005944 1000004085 1000009052 1000010170 1000012993 1000007361
 1000001863 1000002472 1000004303 1000010635 1000010065 1000004085
 1000004085 1000010538 1000005960 1000002604 1000004141 1000010641
 1000014047 1000014238 1000008856 1000009376 1000010151 1000007158
 1000004153 1000015763 1000011448 1000012291 1000014202 1000011435
 1000014202 1000009084 1000012965 1000014202 1000002989 1000005986
 1000015309 1000010136 1000015763 1000010726 1000001921 1000010547
 1000014202 1000005960 1000005850 1000010661 1000010170 1000016662
 1000004085 1000003796 1000002350 1000011349 1000010108 1000021535
 1000004181 1000003228 1000003983 1000008866 1000010538 1000008782
 1000008925 1000004141 1000021535 1000010722 1000019289 1000004085
 1000014252 1000003796 1000002781 10000

INFO:tensorflow:Step: 180 of 12091.
INFO:tensorflow:Validation Speed: 1.305sec/batch
INFO:tensorflow:Roughly  4.317 hours to go.
INFO:tensorflow:Test Label: [1000019530 1000010661 1000008560 1000010683 1000003039 1000001941
 1000010722 1000005960 1000001784 1000018499 1000008094 1000000591
 1000001752 1000006002 1000012991 1000003787 1000003787 1000010001
 1000001863 1000012558 1000014217 1000012949 1000003813 1000002361
 1000011349 1000004307 1000005920 1000014287 1000010170 1000018296
 1000003713 1000018290 1000014202 1000010055 1000010383 1000004591
 1000015309 1000013776 1000010906 1000003796 1000015309 1000008780
 1000015666 1000010100 1000012991 1000018296 1000014202 1000011423
 1000010667 1000011427 1000008958 1000010961 1000014047 1000011429
 1000005274 1000014115 1000002472 1000005960 1000001763 1000002328
 1000010055 1000011427 1000018296 1000003787 1000018296 1000018499
 1000002822 1000002979 1000018296 1000010635 1000010055 1000015309
 1000010641 1000004184 1000004027 10000

INFO:tensorflow:Step: 240 of 12091.
INFO:tensorflow:Validation Speed: 1.342sec/batch
INFO:tensorflow:Roughly  4.419 hours to go.
INFO:tensorflow:Test Label: [1000004141 1000005956 1000010136 1000010551 1000016702 1000012965
 1000008094 1000010170 1000010635 1000001844 1000014238 1000014488
 1000004181 1000009052 1000003275 1000005722 1000005760 1000012447
 1000005326 1000012143 1000004444 1000001831 1000018296 1000018738
 1000015664 1000010053 1000010641 1000012973 1000005928 1000005491
 1000010108 1000018166 1000019772 1000005722 1000018150 1000005956
 1000006016 1000008788 1000012556 1000012983 1000009052 1000003787
 1000010136 1000013387 1000005822 1000010110 1000011401 1000004307
 1000005882 1000008925 1000003952 1000006352 1000005956 1000010170
 1000010110 1000014202 1000003705 1000021533 1000007118 1000010865
 1000017067 1000010108 1000010647 1000003705 1000021533 1000013129
 1000004303 1000012583 1000008770 1000003787 1000004181 1000002177
 1000010170 1000003787 1000000397 10000

...

INFO:tensorflow:Test Ids: [21111562 21111600 21111661 21111735 21111747 21111779 21111804 21111819
 21111832 21111886 21111925 21111976 21112026 21112105 21112146 21112195
 21112255 21112291 21112314 21112336 21112389 21112522 21112558 21112602
 21112660 21112733 21112825 21112859 21112932 21112973 21113021 21113056
 21113099 21113167 21113272 21113330 21113390 21113443 21113482 21113548
 21113601 21113689 21113752 21113771 21113887 21113934 21114033 21114070
 21114144 21114183 21114204 21114217 21114267 21114298 21114363 21114400
 21114487 21114550 21114664 21114684 21114718 21114775 21114803 21114860
 21114900 21114943 21115014 21115046 21115057 21115088 21115146 21115194
 21115251 21115334 21115354 21115388 21115477 21115507 21115543 21115584
 21115610 21115672 21115704 21115772 21115827 21115847 21115978 21116032
 21116085 21116129 21116171 21116212 21116254 21116342 21116420 21116453
 21116507 21116572 21116627 21116653 21116755 21116791 21116835 21116865
 21116969 21117025 211170

INFO:tensorflow:Test Ids: [21639139 21639176 21639312 21639458 21639498 21639582 21639662 21639695
 21639764 21639843 21639900 21639958 21640006 21640062 21640077 21640120
 21640205 21640246 21640262 21640315 21640351 21640377 21640406 21640498
 21640558 21640593 21640663 21640733 21640807 21640882 21640906 21640995
 21641088 21641108 21641135 21641216 21641256 21641354 21641445 21641523
 21641547 21641586 21641645 21641691 21641731 21641753 21641833 21641915
 21641956 21641984 21642029 21642088 21642125 21642172 21642205 21642266
 21642321 21642347 21642411 21642485 21642506 21642584 21642643 21642710
 21642785 21642809 21642868 21642917 21642955 21643030 21643075 21643145
 21643220 21643276 21643324 21643407 21643444 21643482 21643537 21643634
 21643682 21643771 21643804 21643849 21643938 21643968 21644005 21644102
 21644140 21644188 21644225 21644252 21644314 21644369 21644429 21644485
 21644504 21644563 21644663 21644683 21644736 21644793 21644856 21644911
 21644961 21645043 216451

INFO:tensorflow:Test Ids: [22160236 22160303 22160366 22160398 22160462 22160484 22160526 22160605
 22160705 22160761 22160782 22160809 22160873 22160976 22161001 22161022
 22161061 22161085 22161135 22161215 22161260 22161341 22161358 22161417
 22161494 22161543 22161582 22161679 22161773 22161802 22161824 22161862
 22161954 22162009 22162033 22162110 22162228 22162359 22162377 22162441
 22162483 22162519 22162547 22162571 22162609 22162667 22162714 22162765
 22162840 22162914 22162951 22163014 22163086 22163136 22163168 22163214
 22163315 22163353 22163389 22163421 22163463 22163511 22163556 22163609
 22163713 22163790 22163870 22163951 22163967 22164011 22164075 22164115
 22164215 22164246 22164283 22164312 22164365 22164392 22164487 22164527
 22164582 22164626 22164646 22164744 22164777 22164796 22164844 22164975
 22165022 22165073 22165206 22165239 22165372 22165435 22165518 22165540
 22165595 22165638 22165673 22165709 22165769 22165834 22165844 22165874
 22165917 22166004 221660

INFO:tensorflow:Test Ids: [22666116 22666177 22666240 22666307 22666380 22666406 22666481 22666527
 22666557 22666634 22666724 22666795 22666852 22666877 22666925 22666976
 22667043 22667087 22667162 22667247 22667310 22667378 22667418 22667498
 22667591 22667613 22667664 22667679 22667724 22667772 22667841 22667928
 22667985 22668036 22668061 22668110 22668175 22668261 22668282 22668343
 22668465 22668522 22668548 22668587 22668708 22668736 22668823 22668926
 22669033 22669082 22669126 22669160 22669200 22669230 22669283 22669324
 22669393 22669410 22669481 22669523 22669575 22669613 22669689 22669741
 22669780 22669811 22669879 22669917 22669984 22670009 22670045 22670077
 22670109 22670251 22670331 22670453 22670486 22670520 22670558 22670591
 22670669 22670720 22670791 22670844 22670896 22670948 22670978 22671018
 22671059 22671089 22671120 22671168 22671268 22671370 22671411 22671446
 22671499 22671619 22671711 22671759 22671892 22671906 22671948 22671995
 22672036 22672096 226722

INFO:tensorflow:Test Ids: [23184677 23184750 23184875 23184978 23185032 23185051 23185082 23185260
 23185375 23185403 23185461 23185513 23185546 23185601 23185629 23185686
 23185730 23185786 23185871 23185909 23185951 23186035 23186081 23186112
 23186177 23186288 23186330 23186385 23186464 23186539 23186607 23186644
 23186658 23186745 23186804 23186845 23186911 23186943 23187065 23187125
 23187164 23187207 23187254 23187371 23187437 23187494 23187573 23187595
 23187658 23187694 23187755 23187785 23187809 23187856 23187950 23188011
 23188093 23188141 23188232 23188321 23188364 23188413 23188496 23188528
 23188614 23188660 23188680 23188736 23188803 23188831 23188942 23189012
 23189107 23189147 23189210 23189235 23189364 23189445 23189478 23189532
 23189594 23189641 23189678 23189725 23189805 23189863 23189882 23189932
 23190003 23190073 23190153 23190175 23190221 23190243 23190337 23190413
 23190536 23190598 23190650 23190693 23190737 23190806 23190826 23190861
 23190897 23190939 231910

In [6]:
test_data = pd.read_csv(save_file_name)
print(test_data.tail())
# 1768182 
#(1768182, 3095080)
#Test11111 - (700, 1195)

              _id  category_id
1768177  23619792   1000003787
1768178  23619873   1000014224
1768179  23619968   1000010049
1768180  23620195   1000017067
1768181  23620240   1000001756


In [7]:
# print(test_data)